In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

from pprint import pprint as pprint

In [ ]:
# base of configurations
# reference: http://www.gwoptics.org/finesse/examples/aligo_sensitivity.php

basekat = finesse.kat()
basecode = """
# lasers
const Pin 125
const PLO1 1
# spaces
const lx 1.6
const ly 1.55
const Lx 4k
const Ly 4k
const lprc 53
# mirrors
const Tprm 0.03
const Rprm 0.97
const Titm 0.014
const Ritm 0.986
const Tetm 6E-6
const Retm 0.999994
const Tsrm 0.2
const Rsrm 0.8
const Mtm 40 #kg

# carrier laser
l laser $Pin 0 nLaser

# power recycling
m PRM $Rprm $Tprm 90 nLaser nPRM # tuning = 90° towards nLaser
s sBStoPRM $lprc nPRM nBSi

# central beam splitter
bs BS 0.5 0.5 0 45 nBSi nBSr nBSt nBSo
s sBStoYarm $ly nBSr nY0
s sBStoXarm $lx nBSt nX0

# Y arm (perpendicular)
m mYitm $Ritm $Titm 0 nY0 nY1
attr mYitm mass $Mtm
s sY $Ly nY1 nY2
m mYetm $Retm $Tetm 0 nY2 nY3
attr mYetm mass $Mtm

# X arm (parallel)
m mXitm $Ritm $Titm 90 nX0 nX1 # tuning = 90° towards nX0
attr mXitm mass $Mtm
s sX $Lx nX1 nX2
m mXetm $Retm $Tetm 90 nX2 nX3 # tuning = 90° towards nX2
attr mXetm mass $Mtm

# signal recycling
# nBSo to nSRM left disconnected for now
# is connected differently in kat1 and kat2
m SRM $Rsrm $Tsrm 90 nSRM nDark
s sSRMtohdBS 0 nDark nhdBSi # null length space

# (balanced) homodyne detection
# phase of LO s.t. beamsplitter has one constructive, one destructive
l LO1 $PLO1 0 -90 nLO1
bs hdBS 0.5 0.5 0 45 nhdBSi nhdBSr nhdBSt nLO1
qhd qhd1 180 nhdBSr nhdBSt
hd hd1 180 nhdBSr nhdBSt
"""
basekat.parse(basecode)

# pprint((basekat.components, basekat.detectors, basekat.commands))

In [ ]:
def check_tuning(kat):
    """checking IFO tuning by seeing if circulating power in arms is close to 800kW"""
    check_tuning_kat = deepcopy(kat)
    check_tuning_kat.verbose = False
    check_tuning_code="""
    pd initial nLaser*
    pd circPRC nPRM
    pd circX nX1
    pd circY nY1

    pd BSr nBSr
    pd BSt nBSt
    pd BSo nBSo

    noxaxis
    """
    check_tuning_kat.parse(check_tuning_code)
    check_tuning_out = check_tuning_kat.run()

    pprint([(x, '{0:.2f}'.format(check_tuning_out[x].mean())) for x in check_tuning_out.ylabels])    

In [ ]:
def create_kat1(lsrc):
    """configuration 1: connecting nBSo and nSRM with a space
    lsrc in m
    """
    kat1 = deepcopy(basekat)
    kat1code = """
    const lsrc {0}
    s sBStoSRM $lsrc nBSo nSRM
    """.format(lsrc)
    kat1.parse(kat1code)
    
    return kat1


def create_kat2(lsrc, gain=10):
    """configuration 2: connecting nBSo and nSRM with a nle
    lsrc in m, gain in field dB
    """
    kat2 = deepcopy(basekat)
    kat2code = """
    const lsrc {0}
    const halflsrc {1}
    s sBStonle $halflsrc nBSo nnle11
    nle nle1 {2} 0 nnle11 nnle12 # gain defaults to 10 dB, angle = 0
    s snletoSRM $halflsrc nnle12 nSRM  
    """.format(lsrc, lsrc/2, gain)
    kat2.parse(kat2code)
    
    return kat2

In [ ]:
def differential_transfer(kat, fmin=1, fmax=10e9):
    """calculates transfer function for kat1 or kat2
    for differential shaking of arm spaces
    """
    transfer_kat = deepcopy(kat)
    transfer_kat.verbose = False
    # initial frequency of 1 for fsig doesn't matter   
    transfer_code = """          
    fsig darm sX 1 0
    fsig darm sY 1 180 # differential tuning
    
    xaxis darm f log {0} {1} 1000
    yaxis log abs
    """.format(fmin, fmax)
    transfer_kat.parse(transfer_code)
    transfer_out = transfer_kat.run()    
    
    return transfer_out


# def noise_transfer(kat, fmin=1, fmax=10e9):
#     """calculates noise transfer function for kat1 or kat2
#     noise enters through every open port and lossy optic
#     """
#     noisekat = deepcopy(kat)
#     noisekat.verbose = False
#     # initial frequency for fsig doesn't matter   
#     noisecode = """ 
#     fsig noise 100
#    
#     xaxis noise f log {0} {1} 1000
#     yaxis log abs
#     """.format(fmin, fmax)
#     noisekat.parse(noisecode)
#     noiseout = noisekat.run()
#    
#     return noiseout

In [ ]:
class IFO(object):
    def __init__(self, confignum, lsrc, title, filetag):
        """instance of kat1 or kat2
        lsrc is space from central BS to SRM
        title is appended to all plot titles
        filetag is appended to all filenames
        """
        self.lsrc = lsrc
        self.title = title
        self.filetag = filetag+".pdf"
        
        if confignum == 1:
            self.kat = create_kat1(lsrc)
        elif confignum == 2:
            self.kat = create_kat2(lsrc)
        else:
            raise ValueError
            
    def plot_transfer_fns_and_sensitivity(self, fmin=10, fmax=1e4, show=True):
        """plot transfer functions and noise limited sensitivity"""
        plot_out = differential_transfer(self.kat, fmin, fmax)
        plot_x = plot_out.x
        plot_noise = plot_out['qhd1']
        plot_signal = plot_out['hd1']

        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,8))

        ax1.plot(plot_x, plot_noise, label="qhd1")
        ax1.set(title="noise transfer fn - "+self.title,
                xlabel="frequency / Hz", ylabel="noise / au",
                yscale="log", xscale="log")
        ax1.legend()

        ax2.plot(plot_x, plot_signal, label="hd1")
        ax2.set(title="signal transfer fn - "+self.title,
                xlabel="frequency / Hz", ylabel="signal / au",
                yscale="log", xscale="log")
        ax2.legend()

        ax3.plot(plot_x, plot_noise/plot_signal, label="NSR")
        ax3.set(title="QN limited sensitivity - "+self.title,
                xlabel="frequency / Hz", ylabel="strain / ${Hz}^{-1/2}$",
                yscale="log", xscale="log")
        ax3.legend()

        fig.tight_layout()
        fig.savefig("aLIGO_transfer_fns_and_sensitivity"+self.filetag)             

In [ ]:
check_tuning(basekat)

In [ ]:
fmin, fmax = 1e1, 1e4

k11 = IFO(1, 53,   "config. 1, $l_{src}$ = 53 m", "_k11")
k11.plot_transfer_fns_and_sensitivity(fmin=fmin, fmax=fmax)

k12 = IFO(1, 2000, "config. 1, $l_{src}$ = 2 km", "_k12")
k12.plot_transfer_fns_and_sensitivity(fmin=fmin, fmax=fmax)

k21 = IFO(2, 53,   "config. 2, $l_{src}$ = 53 m", "_k21")
k21.plot_transfer_fns_and_sensitivity(fmin=fmin, fmax=fmax)

k22 = IFO(2, 2000, "config. 2, $l_{src}$ = 2 km", "_k22")
k22.plot_transfer_fns_and_sensitivity(fmin=fmin, fmax=fmax)